# Clusterung von Bildern mit dem K-Means-Algorithmus
## Wichtig: hier müssen die Clusterpunkte als Integer definiert sein
 

Quelle:[Lehrtext der ETH-Zürich](http://www.educ.ethz.ch/unterrichtsmaterialien/mathematik/k-means-algorithmus.html)
Hier ein Beispiel:

<table><tr><td><img src='cover_kl.png'></td><td><img src='cover_ausgabe_kl.png'></td></tr>
<tr><td>Ausgangsbild</td><td>Bild komprimiert mit nur 5 Farben</td></tr>
</table>




In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import random
class k_Means():
    """
        Erhaelt eine Punkt-Liste als Array
        die Anzahl der Clusterpunkte
        eventuell die Cluster-Liste, wenn nicht,
        werden die Cluster-Punkte zufaellig aus den
        Punkten der PL bestimmt
    """
    def __init__(self, punkte, anzahl, clLi=None):        
        #print(self.clLi)
        self.puLi = punkte
        self.anzahlCluster = anzahl
        self.laengePuLi, self.dimPunkte =self.puLi.shape
        if clLi:
            self.clLi=clLi
        else:
            self.bestimmeCL()
        print("StartCluster: \n", self.clLi)
        self.cluster= np.zeros((self.laengePuLi,),dtype=int)
        self.error=20
        self.cls=[]
        
    def bestimmeCL(self):
        #zufaellige Auswahl der Clusterpunkte aus den gegebenen
        # MUESSEN INTEGER SEIN
        self.clLi=np.zeros((self.anzahlCluster,self.dimPunkte),dtype=int)
        clListe=[]
        i=0
        while len(clListe) < self.anzahlCluster:
            index= random.randint(0,self.laengePuLi-1)
            cl=self.puLi[index]
            if not self.istEnthalten(clListe,cl): #vermeide Doppelte
                clListe.append(cl)
                self.clLi[i]=self.puLi[index]
                i+=1
                
    def istEnthalten(self,array,el):
        for e in array:
            if (e==el).all():
                return True
        return False
        
    def einteilRunde(self):        
        for i in range(self.laengePuLi):
            #print("Abstaende: ",self.clLi,self.puLi[i])
            abstand=np.linalg.norm(self.clLi-self.puLi[i],axis=1)
            #print(abstand)
            cluster = np.argmin(abstand)
            #print(cluster)
            self.cluster[i]=cluster        
        self.clLi_old=np.copy(self.clLi)
        self.cls.append(self.clLi_old)
        for i in range(self.anzahlCluster):            
            punkte= [self.puLi[j] for j in range(self.laengePuLi) if self.cluster[j]==i]                 
            self.clLi[i]=np.mean(punkte,axis =0)
        print("Neue Cluster: \n", self.clLi)
        self.error = np.linalg.norm(self.clLi-self.clLi_old)
        print(self.error)
        
    def einteilen(self,bis=0):           
        self.einteilRunde()
        while self.error> bis:            
            self.einteilRunde()            
            self.gibZahlen()
            
    def gibZahlen(self):
        print("Verteilung der Punkte auf die Cluster")
        #liefert die Anzahlen der Punkte je Cluster
        for i in range(self.anzahlCluster):            
            punkte= [self.puLi[j] for j in range(self.laengePuLi) if self.cluster[j]==i]
            print(i,len(punkte))
        
    def zeigen(self):
        fig = plt.figure()
        fig.suptitle("Einteilung in Cluster")
        ax = fig.add_subplot(111)        
        farben=["red","yellow","blue","green","lightblue","grey"]*2
        m=["+","x","*","o","v"]*20 #Marker für die Centroiden
        for i in range(self.anzahlCluster):
            xs= [self.puLi[j][0] for j in range(self.laengePuLi) if self.cluster[j]==i]
            ys= [self.puLi[j][1] for j in range(self.laengePuLi) if self.cluster[j]==i]
            #print("xs: ",xs," ys: ",ys)            
            ax.scatter(xs,ys, c=farben[i])        
        si=-1#Markerzaehler
        for c in self.cls:            
            si+=1
            for i in range(self.anzahlCluster):
                ax.scatter(c[i][0],c[i][1], c="black",marker=m[si],label=si)                      
        #for c in self.clLi:
            #ax.scatter(c[0],c[1], c="black",marker=m[0])
        maxis=np.amax(self.puLi,0)
        minis=np.amin(self.puLi,0)
        print("Maxis ",maxis, " Minis: ",minis)
        plt.axis([int(minis[0])-1,int(maxis[0])+1,
                  int(minis[1])-1,int(maxis[1])+1])
        plt.show()

In [2]:
class BildKMeans:
    def __init__(self,datei,anz):
        """
        bekommt den Dateinamen des Bildes und die Anzahl der Farben
        """
        self.datei=datei
        self.anz=anz
        
    def clustern(self,bis=0):
        #wie groß darf die Abweichung sein, default=0
        bk.ppm2array()
        self.k=k_Means(bk.piArray,self.anz)
        self.k.einteilen(bis)        

    def ppm2array(self):
        #einlesen und Array erstellen
        f = open (datei,"r")
        inhalt= f.readlines()
        self.kopf=inhalt[:4]
        groesse=self.kopf[2].strip().split()
        
        self.hoehe=int(groesse[1])
        self.breite=int(groesse[0])
        print(groesse, self.breite, self.hoehe)
        bild=inhalt[4:]
        bli=[]
        for z in bild:
            pl = z.strip().split(" ")
            bli+=pl
        pili=[]
        for i  in range(0,len( bli),3):
            r= int(bli[i])
            g= int(bli[i+1])
            b= int(bli[i+2])
            pili.append([r,g,b])
        self.piArray=np.array(pili) #Startwerte des Bildes speichern
        #print(self.piArray, self.piArray.shape)
        #print(self.kopf)
        
    def array2ppm(self):
        #schreibe zurueck in Datei mit richtigem Format
        kopf=''
        for k in self.kopf:
            kopf+=k
        kopf=kopf[:-1]
        print(kopf)
        ausgabe=self.datei[:-4]+"_ausgabe.ppm"
        bAr=np.copy(self.k.puLi)        
        for i in range(self.k.laengePuLi):
            bAr[i]=self.k.clLi[self.k.cluster[i]]
        bAr=bAr.reshape(self.breite*3,self.hoehe)
        #bAr=np.array2string(bAr)
        print(bAr,bAr.shape)
        np.savetxt(ausgabe, bAr, fmt="%d" ,
                   delimiter=" ",header=kopf,
                   comments='')
        
        

In [3]:
datei="cover.ppm"
bk=BildKMeans(datei,5)
bk.clustern(10)
bk.array2ppm()

['600', '600'] 600 600
StartCluster: 
 [[ 12  19  89]
 [  7  35 220]
 [218   0 122]
 [247   5  55]
 [ 68  52  39]]
Neue Cluster: 
 [[  7  21  65]
 [  6  61 200]
 [223  22 129]
 [218  71  33]
 [ 73 108  61]]
107.54533927604673
Neue Cluster: 
 [[ 19  19  33]
 [  6  56 196]
 [227   9 121]
 [205  76  33]
 [ 95 201 108]]
114.00877159236477
Verteilung der Punkte auf die Cluster
0 93477
1 103051
2 33523
3 74570
4 55379
Neue Cluster: 
 [[ 24  21  26]
 [  6  54 192]
 [227   4 106]
 [201  84  34]
 [ 97 225 118]]
33.301651610693426
Verteilung der Punkte auf die Cluster
0 92685
1 110041
2 40737
3 68470
4 48067
Neue Cluster: 
 [[ 25  22  25]
 [  6  54 191]
 [220   3  89]
 [202 107  33]
 [ 96 227 120]]
29.715315916207253
Verteilung der Punkte auf die Cluster
0 92537
1 111638
2 53961
3 55372
4 46492
Neue Cluster: 
 [[ 26  22  25]
 [  6  54 191]
 [210   5  79]
 [213 140  32]
 [ 95 228 121]]
37.66961640367472
Verteilung der Punkte auf die Cluster
0 92935
1 111850
2 67270
3 42173
4 45772
Neue Cluster: 
